In [31]:
import torch
import torch.nn as nn
from torch.distributions.categorical import Categorical
import tqdm
import matplotlib.pyplot as pp
import numpy as np

In [32]:
INPUT_DIMS = 27
OUTPUT_DIMS = 5
NUM_EPOCHS = 50000
TRAJECTORY_LENGTH = 20
BATCH_SIZE = 10
START_NODE = 25

In [33]:
class TBModel(nn.Module):
  def __init__(self, num_hid):
    super().__init__()
    # The input dimension is 6 for the 6 patches.
    self.mlp = nn.Sequential(nn.Linear(INPUT_DIMS, num_hid), nn.LeakyReLU(),
                             # We now output 12 numbers, 6 for P_F and 6 for P_B
                             nn.Linear(num_hid, OUTPUT_DIMS*2))
    # log Z is just a single number
    self.logZ = nn.Parameter(torch.ones(1))

  def forward(self, x):
    logits = self.mlp(x)
    # Slice the logits, and mask invalid actions (since we're predicting
    # log-values), we use -100 since exp(-100) is tiny, but we don't want -inf)
    P_F = logits[..., :OUTPUT_DIMS] #* (1 - x) + x * -100
    P_B = logits[..., OUTPUT_DIMS:] #* x + (1 - x) * -100
    return P_F, P_B

In [34]:
transitions = {
    1: {0: 1, 1: 2, 2: 1, 3: 3, 4: 1},
    2: {0: 2, 1: 12, 2: 1, 3: 2, 4: 2},
    3: {0: 3, 1: 3, 2: 3, 3: 4, 4: 1},
    4: {0: 4, 1: 5, 2: 4, 3: 6, 4: 3},
    5: {0: 5, 1: 13, 2: 4, 3: 5, 4: 5},
    6: {0: 6, 1: 6, 2: 6, 3: 7, 4: 4},
    7: {0: 7, 1: 8, 2: 7, 3: 9, 4: 6},
    8: {0: 8, 1: 14, 2: 7, 3: 8, 4: 8},
    9: {0: 9, 1: 9, 2: 9, 3: 10, 4: 7},
    10: {0: 10, 1: 11, 2: 10, 3: 10, 4: 9},
    11: {0: 11, 1: 15, 2: 10, 3: 16, 4: 11},
    12: {0: 12, 1: 18, 2: 2, 3: 12, 4: 12},
    13: {0: 13, 1: 19, 2: 5, 3: 13, 4: 13},
    14: {0: 14, 1: 20, 2: 8, 3: 14, 4: 14},
    15: {0: 15, 1: 21, 2: 11, 3: 17, 4: 15},
    16: {0: 16, 1: 17, 2: 16, 3: 16, 4: 11},
    17: {0: 17, 1: 22, 2: 16, 3: 23, 4: 15},
    18: {0: 18, 1: 18, 2: 12, 3: 25, 4: 18},
    19: {0: 19, 1: 19, 2: 13, 3: 26, 4: 25},
    20: {0: 20, 1: 20, 2: 14, 3: 27, 4: 26},
    21: {0: 21, 1: 21, 2: 15, 3: 22, 4: 27},
    22: {0: 22, 1: 22, 2: 17, 3: 24, 4: 21},
    23: {0: 23, 1: 24, 2: 17, 3: 23, 4: 17},
    24: {0: 24, 1: 22, 2: 23, 3: 24, 4: 22},
    25: {0: 25, 1: 25, 2: 25, 3: 19, 4: 18},
    26: {0: 26, 1: 26, 2: 26, 3: 20, 4: 19},
    27: {0: 27, 1: 27, 2: 27, 3: 21, 4: 20}
}
def state_to_vec(state):
    result = [0]*27
    result[state-1] = 1
    return torch.tensor(result).float()

def reward(state):
    if state == 10:
        return 2
    if state == 2:
        return 1
    return 0

In [36]:
# Instantiate model and optimizer
model = TBModel(512)
opt = torch.optim.Adam(model.parameters(),  3e-4)

# Let's keep track of the losses and the faces we sample
tb_losses = []
tb_sampled_faces = []
tb_rewards = []
logZs = []

# To not complicate the code, I'll just accumulate losses here and take a
# gradient step every `update_freq` episode.
minibatch_loss = 0
minibatch_reward = 0

for episode in tqdm.tqdm(range(NUM_EPOCHS), ncols=40):
  # Each episode starts with an "empty state"
  state = START_NODE
  # Predict P_F, P_B
  P_F_s, P_B_s = model(state_to_vec(state))
  total_P_F = 0
  total_P_B = 0
  for t in range(TRAJECTORY_LENGTH):
    # Here P_F is logits, so we want the Categorical to compute the softmax for us
    cat = Categorical(logits=P_F_s)
    action = cat.sample()
    # "Go" to the next state
    new_state = transitions[state][action.item()]
    # Accumulate the P_F sum
    total_P_F += cat.log_prob(action)

    if t == TRAJECTORY_LENGTH-1:
      # If we've built a complete face, we're done, so the reward is > 0
      # (unless the face is invalid)
      reward = torch.tensor(reward(new_state))
    # We recompute P_F and P_B for new_state
    P_F_s, P_B_s = model(state_to_vec(new_state))
    # Here we accumulate P_B, going backwards from `new_state`. We're also just
    # going to use opposite semantics for the backward policy. I.e., for P_F action
    # `i` just added the face part `i`, for P_B we'll assume action `i` removes
    # face part `i`, this way we can just keep the same indices.
    total_P_B += Categorical(logits=P_B_s).log_prob(action)

    # Continue iterating
    state = new_state

  # We're done with the trajectory, let's compute its loss. Since the reward can
  # sometimes be zero, instead of log(0) we'll clip the log-reward to -20.
  loss = (model.logZ + total_P_F - torch.log(reward).clip(-20) - total_P_B).pow(2)
  minibatch_loss += loss
  minibatch_reward += reward


  # Add the face to the list, and if we are at an
  # update episode, take a gradient step.
  tb_sampled_faces.append(state)
  if episode % BATCH_SIZE == 0:
    tb_losses.append(minibatch_loss.item())
    tb_rewards.append(minibatch_reward.item())
    minibatch_loss.backward()
    opt.step()
    opt.zero_grad()
    minibatch_loss = 0
    minibatch_reward = 0
    logZs.append(model.logZ.item())


  0%|         | 0/50000 [00:00<?, ?it/s]

new_state 18
new_state 18
new_state 18


TypeError: 'Tensor' object is not callable

In [ ]:
f, ax = pp.subplots(3, 1, figsize=(10,6))
pp.sca(ax[0])
pp.plot(tb_losses)
pp.yscale('log')
pp.ylabel('loss')
pp.sca(ax[1])
pp.plot(np.exp(logZs))
pp.ylabel('estimated Z')
pp.sca(ax[2])
pp.plot(tb_rewards)
pp.ylabel('rewards')